In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import functions

import glob
import pandas as pd
import re
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

import matplotlib.colors as mcolors

import numpy as np

### df vs ff, daily vs once, langtang vs mustang -> 4 rows total 


In [30]:
# functions 

start_date = "1990-07-31"
end_date = "2021-06-30"


def filter_by_date(df, start_date = start_date, end_date = end_date, date_column='D'):
    """
    Filters a DataFrame to include only rows where the date_column is within the given range.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        start_date (str or pd.Timestamp): The start date (inclusive).
        end_date (str or pd.Timestamp): The end date (inclusive).
        date_column (str): The column containing date values (default is 'D').

    Returns:
        pd.DataFrame: The filtered DataFrame.
    """
    df[date_column] = pd.to_datetime(df[date_column])  # Convert to datetime
    return df[(df[date_column] >= start_date) & (df[date_column] <= end_date)]


# Define a function to categorize seasons
def get_season(month):
    if month in [12, 1, 2]:
        return 'DJF'  # December, January, February (Winter)
    elif month in [3, 4, 5]:
        return 'MAM'  # March, April, May (Spring)
    elif month in [6, 7, 8]:
        return 'JJA'  # June, July, August (Summer)
    else:
        return 'SON'  # September, October, November (Autumn)

# Define a function to categorize seasons
def get_monsoon(month):
    if month in [1, 2, 3, 4]:
        return 'before monsoon'  
    elif month in [5, 6, 7, 8, 9]:
        return month  # should remain the same 
    elif month in [10, 11, 12]:
        return 'after monsoon'  # June, July, August (Summer)






def prepare_dfcount_for_plot(dfcount):
    
    dfcount = dfcount.drop('folder', axis = 1)
    
    melted = pd.melt(dfcount, id_vars=['D', 'year', 'month'], var_name='elevation', value_name='dfs_count')
    melted['elevation'] = melted['elevation'].str.split('.').str[0].astype(float)

    melted['id'] = melted['D_year'].astype(str)+ "_" + melted['D_month'].astype(str) + "_" + melted['elevation'].astype(str)
    
    melted['elevation_bin'] = melted.apply(functions.bin_elevation500, axis=1)
    melted = melted.sort_values('elevation_bin')
    # melted = melted.dropna(subset=['dfs_count'])
    return melted 







def prepare_dfcount_for_barplot(dfcount, landcover_idx):
    melted = pd.melt(dfcount, id_vars=['D', 'D_year', 'D_month'], var_name='elevation', value_name='dfs_count')
    melted['elevation'] = melted['elevation'].str.split('.').str[0].astype(float)
    melted['elevation_bin'] = melted.apply(functions.bin_elevation500, axis=1)
   
    # melted['id'] = melted['D_year'].astype(str)+ "_" + melted['D_month'].astype(str) + "_" + melted['elevation'].astype(str)

    melted = melted.sort_values('elevation_bin')
    melted['season'] = melted['D_month'].apply(get_season)
    melted['monsoon'] = melted['D_month'].apply(get_monsoon)

    melted['landcover'] = f'landcover {landcover_idx}'
    return melted 



def merge_landcover_dfs(dfs, dfspot, merge_on, dfspot_column):
    """
    Merges a list of dataframes on specified columns and adds a column from a 'dfspot' dataframe.
    
    Parameters:
        dfs (list of pd.DataFrame): List of dataframes to merge sequentially.
        dfspot (pd.DataFrame): A dataframe containing a column to add after the merges.
        merge_on (list of str): Columns to use as the merge keys.
        dfspot_column (str): The name of the column in `dfspot` to add after merging.
        
    Returns:
        pd.DataFrame: The merged dataframe with the additional column from `dfspot`.
    """
    
    # Start with the first dataframe in `dfs` and automatically get the count column
    count_column = [col for col in dfs[0].columns if col.startswith('dfs_count')][0]
    merged_df = dfs[0][merge_on + [count_column]]
    
    # Iterate over remaining dataframes and merge each sequentially
    for df in dfs[1:]:
        count_column = [col for col in df.columns if col.startswith('dfs_count')][0]
        merged_df = pd.merge(merged_df, df[merge_on + [count_column]], on=merge_on)
    
    # Add the `dfspot_column` from `dfspot` dataframe to the merged dataframe
    merged_df['dfspot_count'] = dfspot[dfspot_column]
    
    return merged_df


def calculate_floods(df):
    # df percent 
    df[f'dfs_count_60_percent'] = (df[f'dfs_count_60percent'] *100) / df.dfspot_count
    df[f'dfs_count_50_percent'] = (df[f'dfs_count_50percent'] *100) / df.dfspot_count
    df[f'dfs_count_40_percent'] = (df[f'dfs_count_40percent'] *100) / df.dfspot_count
    df[f'dfs_count_30_percent'] = (df[f'dfs_count_30percent'] *100) / df.dfspot_count
    df[f'dfs_count_20_percent'] = (df[f'dfs_count_20percent'] *100) / df.dfspot_count
    # ff count 
    df[f'ffs_count_60'] = df.dfspot_count - df[f'dfs_count_60percent']
    df[f'ffs_count_50'] = df.dfspot_count - df[f'dfs_count_50percent']
    df[f'ffs_count_40'] = df.dfspot_count - df[f'dfs_count_40percent']
    df[f'ffs_count_30'] = df.dfspot_count - df[f'dfs_count_30percent']
    df[f'ffs_count_20'] = df.dfspot_count - df[f'dfs_count_20percent']
    # ff percent
    df[f'ffs_count_60_percent'] = (df[f'ffs_count_60'] *100) / df.dfspot_count
    df[f'ffs_count_50_percent'] = (df[f'ffs_count_50'] *100) / df.dfspot_count
    df[f'ffs_count_40_percent'] = (df[f'ffs_count_40'] *100) / df.dfspot_count
    df[f'ffs_count_30_percent'] = (df[f'ffs_count_30'] *100) / df.dfspot_count
    df[f'ffs_count_20_percent'] = (df[f'ffs_count_20'] *100) / df.dfspot_count
    return df



# mustang - potential


In [28]:
location = 'mustang'

# path = '/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2025Jan_output/TL/langtang/'
path = '/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2025Jan_output/TL/march2025mustang/'
dfspot1_df = pd.read_csv(path + f'{location}_monthly_dfspot_count_landcover1.csv', index_col = 0).fillna(0)
dfspot2_df = pd.read_csv(path + f'{location}_monthly_dfspot_count_landcover2.csv', index_col = 0).fillna(0)
dfspot3_df = pd.read_csv(path + f'{location}_monthly_dfspot_count_landcover3.csv', index_col = 0).fillna(0)
dfspot4_df = pd.read_csv(path + f'{location}_monthly_dfspot_count_landcover4.csv', index_col = 0).fillna(0)
dfspot5_df = pd.read_csv(path + f'{location}_monthly_dfspot_count_landcover5.csv', index_col = 0).fillna(0)

dfspot1_df = filter_by_date(dfspot1_df)
dfspot2_df = filter_by_date(dfspot2_df)
dfspot3_df = filter_by_date(dfspot3_df)
dfspot4_df = filter_by_date(dfspot4_df)
dfspot5_df = filter_by_date(dfspot5_df)

dfspot1 = prepare_dfcount_for_barplot(dfspot1_df, 1).fillna(0)
dfspot2 = prepare_dfcount_for_barplot(dfspot2_df, 2).fillna(0)
dfspot3 = prepare_dfcount_for_barplot(dfspot3_df, 3).fillna(0)
dfspot4 = prepare_dfcount_for_barplot(dfspot4_df, 4).fillna(0)
dfspot5 = prepare_dfcount_for_barplot(dfspot5_df, 5).fillna(0)

dfspot1 = dfspot1.rename(columns = {'D_year':'year', 'D_month':'month'})
dfspot2 = dfspot2.rename(columns = {'D_year':'year', 'D_month':'month'})
dfspot3 = dfspot3.rename(columns = {'D_year':'year', 'D_month':'month'})
dfspot4 = dfspot4.rename(columns = {'D_year':'year', 'D_month':'month'})
dfspot5 = dfspot5.rename(columns = {'D_year':'year', 'D_month':'month'})

In [29]:
dfspot3

,D,year,month,elevation,dfs_count,elevation_bin,season,monsoon,landcover
1765,2013-08-31,2013,8,2886.0,0.0,2500 - 3000,JJA,8,landcover 3
1729,2010-08-31,2010,8,2886.0,0.0,2500 - 3000,JJA,8,landcover 3
1730,2010-09-30,2010,9,2886.0,0.0,2500 - 3000,SON,9,landcover 3
1731,2010-10-31,2010,10,2886.0,0.0,2500 - 3000,SON,after monsoon,landcover 3
1732,2010-11-30,2010,11,2886.0,0.0,2500 - 3000,SON,after monsoon,landcover 3
...,...,...,...,...,...,...,...,...,...
31006,2001-05-31,2001,5,6066.0,0.0,6000,MAM,5,landcover 3
31005,2001-04-30,2001,4,6066.0,0.0,6000,MAM,before monsoon,landcover 3
31004,2001-03-31,2001,3,6066.0,0.0,6000,MAM,before monsoon,landcover 3
31014,2002-01-31,2002,1,6066.0,0.0,6000,DJF,before monsoon,landcover 3


In [22]:
# dfspot3.date_id.unique().shape

# once and daily

In [23]:

landcover_idx = 1
method = 'daily'
location = 'mustang'



# Base directory where all output_Xpercent folders are located
base_directory = f"/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2025Jan_output/SL_{method}/"

# Use glob to find all relevant files in output_XXpercent folders
file_pattern = f"{base_directory}/output_*/langtang_monthly_dfs_count_*_landcover{landcover_idx}.csv"
files = glob.glob(file_pattern)

dfs = []

# Loop through the files and process them
for file in files:
    # Extract the percentile from the folder name
    percentile = re.search(r'output_(\d+percent)', file).group(1)
    
    # Extract the landcover number from the file name
    landcover_number = re.search(r'landcover(\d+)', file).group(1)
    print(landcover_number)
    print(file)
    # Read the file into a DataFrame
    df = pd.read_csv(file, index_col=0).fillna(0)
    df = filter_by_date(df)

    # Apply your processing function
    processed_df = prepare_dfcount_for_plot(df)
    
    # Rename the 'dfs_count' column to include percentile and landcover number
    processed_df = processed_df.rename(columns={'dfs_count': f'dfs_count_{percentile}'})
    # Append to the list
    dfs.append(processed_df)

merge_on = ['year', 'month', 'elevation', 'elevation_bin', 'id']
dfspot_column = 'dfs_count'

if landcover_idx == 1:
    dfspot = dfspot1
if landcover_idx == 2:
    dfspot = dfspot2
if landcover_idx == 3:
    dfspot = dfspot3
if landcover_idx == 4:
    dfspot = dfspot4
if landcover_idx == 5:
    dfspot = dfspot5


dfmerged = merge_landcover_dfs(dfs, dfspot, merge_on, dfspot_column)
dffloods = calculate_floods(dfmerged)


outpath = '/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2025Jan_output/df_vs_floods/mustang_test/'
# dffloods.to_csv(outpath + f'{location}_df_vs_floods_{method}_landcover{landcover_idx}.csv')


1
/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2025Jan_output/SL_daily/output_30percent/langtang_monthly_dfs_count_30percent_landcover1.csv
1
/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2025Jan_output/SL_daily/output_20percent/langtang_monthly_dfs_count_20percent_landcover1.csv
1
/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2025Jan_output/SL_daily/output_60percent/langtang_monthly_dfs_count_60percent_landcover1.csv
1
/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2025Jan_output/SL_daily/output_40percent/langtang_monthly_dfs_count_40percent_landcover1.csv
1
/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2025Jan_output/SL_daily/output_50percent/langtang_monthly_dfs_count_50percent_landcover1.csv


KeyError: "['year', 'month', 'id'] not in index"

In [32]:
test = pd.read_csv('/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2025Jan_output/SL_once/output_20percent/mustang_monthly_sum_elevation_dfs_20percent_1landcover_mm.csv')
test

,month,year,3689.0,2546.0,3964.0,2571.0,2886.0,3795.0,3540.0,4420.0,...,4785.0,4439.0,4593.0,4603.0,4733.0,4905.0,4952.0,4812.0,4889.0,folder
0,9,1989,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mustang_climate_cut
1,10,1989,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mustang_climate_cut
2,11,1989,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mustang_climate_cut
3,12,1989,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mustang_climate_cut
4,1,1990,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mustang_climate_cut
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,5,2022,0.000000,3.726750,0.000000,3.708209,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mustang_climate_cut
393,6,2022,58.594102,132.535223,54.037074,132.259252,61.169990,37.680373,46.224046,37.695656,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mustang_climate_cut
394,7,2022,193.335784,353.241563,175.066199,345.650127,345.187199,201.240182,227.389835,155.959179,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mustang_climate_cut
395,8,2022,17.996708,69.527600,14.075152,73.409143,142.442272,67.632671,107.280142,40.589535,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mustang_climate_cut


In [31]:
test = prepare_dfcount_for_plot(test)


KeyError: "The following 'id_vars' are not present in the DataFrame: ['D']"

# plots with bubbles 

In [ ]:
# read those files 
outpath = '/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2025Jan_output/df_vs_floods/'

landcover1O = pd.read_csv(outpath + 'langtang_df_vs_floods_once_landcover1.csv', index_col=0)
landcover2O = pd.read_csv(outpath + 'langtang_df_vs_floods_once_landcover2.csv', index_col=0)
landcover3O = pd.read_csv(outpath + 'langtang_df_vs_floods_once_landcover3.csv', index_col=0)
landcover4O = pd.read_csv(outpath + 'langtang_df_vs_floods_once_landcover4.csv', index_col=0)
landcover5O = pd.read_csv(outpath + 'langtang_df_vs_floods_once_landcover5.csv', index_col=0)


landcover1D = pd.read_csv(outpath + 'langtang_df_vs_floods_daily_landcover1.csv', index_col=0)
landcover2D = pd.read_csv(outpath + 'langtang_df_vs_floods_daily_landcover2.csv', index_col=0)
landcover3D = pd.read_csv(outpath + 'langtang_df_vs_floods_daily_landcover3.csv', index_col=0)
landcover4D = pd.read_csv(outpath + 'langtang_df_vs_floods_daily_landcover4.csv', index_col=0)
landcover5D = pd.read_csv(outpath + 'langtang_df_vs_floods_daily_landcover5.csv', index_col=0)


In [ ]:
# # check the "empty" elevation bins without any DFs potentially 

# dfs = {"df1": landcover1, "df2": landcover2, "df3": landcover3, "df4": landcover4, "df5": landcover5}  # Example dictionary of dataframes

# empty_bins_per_df = {}

# for name, df in dfs.items():
#     # empty_bins = df.groupby("elevation_bin")["dfspot_count"].apply(lambda x: (x.isna() | (x == 0)).all())
#     empty_bins = df.groupby("elevation_bin")["dfspot_count"].apply(lambda x: (x.isna()).all())
#     empty_bins_per_df[name] = empty_bins[empty_bins].index.tolist()

# # Print results
# for name, empty_bins in empty_bins_per_df.items():
#     print(f"Elevation bins with only 0/NaN dfspot in {name}: {empty_bins}")


In [ ]:
# def monthly_mean_per_elevation(df):
#     mean = df.groupby(['elevation_bin', 'month']).mean().reset_index() 
#     return mean
    
def monthly_mean(df):
    mean = df.groupby(['month']).mean().reset_index() 
    return mean


# landcover1monthly_comb = monthly_mean(landcover1)
# landcover2monthly_comb = monthly_mean(landcover2)
# landcover3monthly_comb = monthly_mean(landcover3)
# landcover4monthly_comb = monthly_mean(landcover4)
# landcover5monthly_comb = monthly_mean(landcover5)


landcover1monthlyO = monthly_mean(landcover1O)
landcover2monthlyO = monthly_mean(landcover2O)
landcover3monthlyO = monthly_mean(landcover3O)
landcover4monthlyO = monthly_mean(landcover4O)
landcover5monthlyO = monthly_mean(landcover5O)

landcover1monthlyO['landcover']='landcover 1'
landcover2monthlyO['landcover']='landcover 2'
landcover3monthlyO['landcover']='landcover 3'
landcover4monthlyO['landcover']='landcover 4'
landcover5monthlyO['landcover']='landcover 5'

landcover1monthlyD = monthly_mean(landcover1D)
landcover2monthlyD = monthly_mean(landcover2D)
landcover3monthlyD = monthly_mean(landcover3D)
landcover4monthlyD = monthly_mean(landcover4D)
landcover5monthlyD = monthly_mean(landcover5D)


landcover1monthlyD['landcover']='landcover 1'
landcover2monthlyD['landcover']='landcover 2'
landcover3monthlyD['landcover']='landcover 3'
landcover4monthlyD['landcover']='landcover 4'
landcover5monthlyD['landcover']='landcover 5'

landcover_all_month_daily = pd.concat([landcover1monthlyD, landcover2monthlyD, landcover3monthlyD, 
                                       landcover4monthlyD, landcover5monthlyD])

landcover_all_month_once = pd.concat([landcover1monthlyO, landcover2monthlyO, landcover3monthlyO, 
                                      landcover4monthlyO, landcover5monthlyO])

In [ ]:
testdaily1= landcover_all_month_daily[landcover_all_month_daily['landcover'].str.strip() == 'landcover 5']
testonce1= landcover_all_month_once[landcover_all_month_once['landcover'].str.strip() ==    'landcover 5']


In [ ]:
fig = plt.figure(figsize=(20, 5), layout='tight')
mosaic = fig.subplot_mosaic('''
                            abcde
                            ABCDE
                            ''')


sns.barplot(data = testdaily1, y = 'dfs_count_20_percent', x='month', ax = mosaic['a'], hue = 'dfspot_count')
sns.barplot(data = testdaily1, y = 'dfs_count_30_percent', x='month', ax = mosaic['b'], hue = 'dfspot_count')#, hue = 'elevation_bin',, legend = False)
sns.barplot(data = testdaily1, y = 'dfs_count_40_percent', x='month', ax = mosaic['c'], hue = 'dfspot_count')#, hue = 'elevation_bin',, legend = False)
sns.barplot(data = testdaily1, y = 'dfs_count_50_percent', x='month', ax = mosaic['d'], hue = 'dfspot_count')
sns.barplot(data = testdaily1, y = 'dfs_count_60_percent', x='month', ax = mosaic['e'], hue = 'dfspot_count')

sns.barplot(data = testonce1, y = 'dfs_count_20_percent', x='month', ax = mosaic['A'], hue = 'dfspot_count')
sns.barplot(data = testonce1, y = 'dfs_count_30_percent', x='month', ax = mosaic['B'], hue = 'dfspot_count')#, hue = 'elevation_bin',, legend = False)
sns.barplot(data = testonce1, y = 'dfs_count_40_percent', x='month', ax = mosaic['C'], hue = 'dfspot_count')#, hue = 'elevation_bin',, legend = False)
sns.barplot(data = testonce1, y = 'dfs_count_50_percent', x='month', ax = mosaic['D'], hue = 'dfspot_count')
sns.barplot(data = testonce1, y = 'dfs_count_60_percent', x='month', ax = mosaic['E'], hue = 'dfspot_count')


In [ ]:
# landcover_all_month_daily.columns
# which columns do i care about?

# ffs_count_60_percent
# dfs_count_60_percent
# dfspot_count

'''
'dfspot_count', 
'dfs_count_60_percent',
'dfs_count_50_percent', 
'dfs_count_40_percent', 
'dfs_count_30_percent',
'dfs_count_20_percent', 
'ffs_count_60_percent',
'ffs_count_50_percent', 
'ffs_count_40_percent', 
'ffs_count_30_percent',
'ffs_count_20_percent', 
'landcover'
'''
# landcover_all_month_daily

In [ ]:
daily20 = landcover_all_month_daily[['dfs_count_20_percent','ffs_count_20_percent', 'dfspot_count', 'landcover', 'month','year',	'elevation']]
daily30 = landcover_all_month_daily[['dfs_count_30_percent','ffs_count_30_percent', 'dfspot_count', 'landcover', 'month','year',	'elevation']]
daily40 = landcover_all_month_daily[['dfs_count_40_percent','ffs_count_40_percent', 'dfspot_count', 'landcover', 'month','year',	'elevation']]
daily50 = landcover_all_month_daily[['dfs_count_50_percent','ffs_count_50_percent', 'dfspot_count', 'landcover', 'month','year',	'elevation']]
daily60 = landcover_all_month_daily[['dfs_count_60_percent','ffs_count_60_percent', 'dfspot_count', 'landcover', 'month','year',	'elevation']]

once20 = landcover_all_month_once[['dfs_count_20_percent','ffs_count_20_percent', 'dfspot_count', 'landcover', 'month','year',	'elevation']]
once30 = landcover_all_month_once[['dfs_count_30_percent','ffs_count_30_percent', 'dfspot_count', 'landcover', 'month','year',	'elevation']]
once40 = landcover_all_month_once[['dfs_count_40_percent','ffs_count_40_percent', 'dfspot_count', 'landcover', 'month','year',	'elevation']]
once50 = landcover_all_month_once[['dfs_count_50_percent','ffs_count_50_percent', 'dfspot_count', 'landcover', 'month','year',	'elevation']]
once60 = landcover_all_month_once[['dfs_count_60_percent','ffs_count_60_percent', 'dfspot_count', 'landcover', 'month','year',	'elevation']]


In [ ]:
import pandas as pd

# Define a function to rename columns
def rename_and_add_column(df, percentile):
    df = df.rename(columns={
        df.columns[0]: 'dfs_count_percent',  # Renaming first column
        df.columns[1]: 'ffs_count_percent'   # Renaming second column
    })
    df['percentile_input'] = f'{percentile} percent'
    return df

# Rename and add column for each dataframe
daily20 = rename_and_add_column(daily20, 20)
daily30 = rename_and_add_column(daily30, 30)
daily40 = rename_and_add_column(daily40, 40)
daily50 = rename_and_add_column(daily50, 50)
daily60 = rename_and_add_column(daily60, 60)

# Concatenate all dataframes
df_daily = pd.concat([daily20, daily30, daily40, daily50, daily60], ignore_index=True)

df_daily1= df_daily[df_daily.landcover =='landcover 1']
df_daily2= df_daily[df_daily.landcover =='landcover 2']
df_daily3= df_daily[df_daily.landcover =='landcover 3']
df_daily4= df_daily[df_daily.landcover =='landcover 4']
df_daily5= df_daily[df_daily.landcover =='landcover 5']


# Rename and add column for each dataframe
once20 = rename_and_add_column(once20, 20)
once30 = rename_and_add_column(once30, 30)
once40 = rename_and_add_column(once40, 40)
once50 = rename_and_add_column(once50, 50)
once60 = rename_and_add_column(once60, 60)

# Concatenate all dataframes
df_once = pd.concat([once20, once30, once40, once50, once60], ignore_index=True)

df_once1= df_once[df_once.landcover =='landcover 1']
df_once2= df_once[df_once.landcover =='landcover 2']
df_once3= df_once[df_once.landcover =='landcover 3']
df_once4= df_once[df_once.landcover =='landcover 4']
df_once5= df_once[df_once.landcover =='landcover 5']




# final plot for langtang! 

In [ ]:
vmin = 0
vmax = 100
center = vmax/2
# Convert categorical landcover to numeric for plotting
percentile_numeric, percentile_labels = pd.factorize(df_once1.percentile_input)

# # Define normalization with midpoint at 50%
norm = mcolors.TwoSlopeNorm(vmin=vmin, vcenter=center, vmax=vmax)



fig = plt.figure(figsize=(20, 5.8), layout='tight')
mosaic = fig.subplot_mosaic('''
                            ABCDE
                            abcde
                            abcde
                            abcde
                            abcde
                            abcde
                            abcde
                            ''')

# List of subplot keys where the grey background should be applied
target_axes = ['a', 'b', 'c', 'd', 'e']

# Apply axvspan **only to the target axes**
for key in target_axes:
    mosaic[key].axvspan(5 - 0.5, 9 + 0.5, color='grey', alpha=0.15)

s1 = mosaic['a'].scatter(x = df_daily1.month, y = percentile_numeric-0.20, c = df_daily1.dfs_count_percent, s = df_daily1.dfspot_count*100, cmap = 'coolwarm', norm=norm, edgecolor = 'none')
mosaic['a'].scatter(x = df_once1.month, y = percentile_numeric+0.20, c = df_once1.dfs_count_percent, s = df_once1.dfspot_count*100, cmap = 'coolwarm', norm=norm, edgecolor = 'none')

mosaic['b'].scatter(x = df_daily2.month, y = percentile_numeric-0.20, c = df_daily2.dfs_count_percent, s = df_daily2.dfspot_count*100, cmap = 'coolwarm', norm=norm,  edgecolor = 'none')
mosaic['b'].scatter(x = df_once2.month,  y = percentile_numeric+0.20,  c = df_once2.dfs_count_percent,  s = df_once2.dfspot_count*100, cmap = 'coolwarm', norm=norm)

mosaic['c'].scatter(x = df_daily3.month, y = percentile_numeric-0.20, c = df_daily3.dfs_count_percent, s = df_daily3.dfspot_count*100, cmap = 'coolwarm', norm=norm,  edgecolor = 'none')
mosaic['c'].scatter(x = df_once3.month,  y = percentile_numeric+0.20,  c = df_once3.dfs_count_percent,  s = df_once3.dfspot_count*100, cmap = 'coolwarm', norm=norm)

mosaic['d'].scatter(x = df_daily4.month, y = percentile_numeric-0.20, c = df_daily4.dfs_count_percent, s = df_daily4.dfspot_count*100, cmap = 'coolwarm', norm=norm,  edgecolor = 'none')
mosaic['d'].scatter(x = df_once4.month,  y = percentile_numeric+0.20,  c = df_once4.dfs_count_percent,  s = df_once4.dfspot_count*100, cmap = 'coolwarm', norm=norm)


mosaic['e'].scatter(x = df_daily5.month, y = percentile_numeric-0.20, c = df_daily5.dfs_count_percent, s = df_daily5.dfspot_count*100, cmap = 'coolwarm', norm=norm,  edgecolor = 'none')
mosaic['e'].scatter(x = df_once5.month,  y = percentile_numeric+0.20,  c = df_once5.dfs_count_percent,  s = df_once5.dfspot_count*100, cmap = 'coolwarm', norm=norm)


mosaic['a'].set_yticks(range(len(percentile_labels)))
mosaic['a'].set_yticklabels(percentile_labels)



# cbar1 = fig.colorbar(s1, ax=mosaic['e'], orientation='vertical', fraction=0.1, pad=0.1)


# -------------------

for key in ['a', 'b', 'c', 'd', 'e']:
    mosaic[key].set_xticks(df_once1.month.unique())  # Set tick positions
    mosaic[key].set_xticklabels(df_once1.month.unique(), rotation=45)  # Set tick labels, rotate

for key in ['a']:
    mosaic[key].set_yticks(range(len(percentile_labels)))
    mosaic[key].set_yticklabels(percentile_labels)

for key in ['b', 'c', 'd', 'e']:
    mosaic[key].set_yticklabels([])


mosaic['A'].set_title('landcover 1')
mosaic['B'].set_title('landcover 2')
mosaic['C'].set_title('landcover 3')
mosaic['D'].set_title('landcover 4')
mosaic['E'].set_title('landcover 5')


# plt.savefig(outpath + 'bubble_plot_landcover_panels_new_legend.png', dpi = 300, bbox_inches = 'tight')
plt.suptitle('langtang, df count across all elevations', fontsize = 20)


# color bar for land cover shares 

# Define color proportions for each panel (Green, Yellow, Gray)
shares = [
    (40, 40, 20),  # First bar: 40% green (veg) , 30% grey (bedrock), 30% blue (glacier)
    (40, 50, 10),
    (50, 40, 10),
    (40, 60, 00),
    (60, 40, 00),
]

# Iterate over the bar row and create the proportional bars
for i, key in enumerate("ABCDE"):
    bar_ax = mosaic[key]  # Select the correct subplot for the bar

    # Remove ticks, labels, and spines
    bar_ax.set_xticks([])
    bar_ax.set_yticks([])
    bar_ax.set_frame_on(False)

    # Extract proportions (must sum to 100%)
    p1, p2, p3 = shares[i]  # Green, Yellow, Gray

    # Convert percentages to width (normalized to 1)
    p1 /= 100
    p2 /= 100
    p3 /= 100

    # Plot the three segments
    bar_ax.barh(0, p1, height=1, color='green', align='center', alpha = 0.8)
    bar_ax.barh(0, p2, height=1, color='lightgray', left=p1, align='center')
    bar_ax.barh(0, p3, height=1, color='lightblue', left=p1 + p2, align='center')

    # # Add percentage label (can be adjusted)
    # bar_ax.text(0.5, 0, f"{shares[i][0]}% / {shares[i][1]}% / {shares[i][2]}%", 
    #             ha='center', va='center', fontsize=12, color='black', weight='bold')


# plt.savefig('/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/output/testplots/bubble_plot_landcover_panels_new_legend.png', bbox_inches = 'tight', dpi = 300)





In [ ]:
outpath

In [ ]:
# df_daily1

In [ ]:
df_once1[df_once1['percentile_input'] =='20 percent'].dfs_count_percent.plot(label = 'once')
df_daily1[df_daily1['percentile_input'] =='20 percent'].dfs_count_percent.plot(label = 'daily')
plt.hline(
plt.legend()

In [ ]:
df_once1.dfs_count_percent.plot()
df_daily1.dfs_count_percent.plot()

In [ ]:
df_once2.dfs_count_percent.plot()
df_daily2.dfs_count_percent.plot()